## Test Match Dataset

In [1]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import bar_chart_race as bcr

In [2]:
def extract_match_results(year):
    #URL = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id=2012;type=year'
    URL = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=1;id=' + year + ';type=year'
    page = requests.get(URL)
    bs = BeautifulSoup(page.content, 'lxml')
    table_body=bs.find_all('tbody')
    li = []
    rows = table_body[0].find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        li.append([cols[2].string, cols[5].string])
    return li

In [3]:
final_list = []
for year in range(1877,2022):
    try:
        li = extract_match_results(str(year))
        for elem in li:
            final_list.append(elem)
    except Exception as e:
        pass

In [4]:
df = pd.DataFrame(final_list, columns = ['Team', 'Date'])

In [5]:
df.head()

,Team,Date
0,Australia,"Mar 15-19, 1877"
1,England,"Mar 31-Apr 4, 1877"
2,Australia,"Jan 2-4, 1879"
3,England,"Sep 6-8, 1880"
4,drawn,"Dec 31, 1881 - Jan 4, 1882"


In [6]:
df.shape

(2479, 2)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Team    2479 non-null   object
 1   Date    2479 non-null   object
dtypes: object(2)
memory usage: 38.9+ KB


In [8]:
df['Year'] = df['Date'].str[-4:]

In [9]:
df.head()

,Team,Date,Year
0,Australia,"Mar 15-19, 1877",1877
1,England,"Mar 31-Apr 4, 1877",1877
2,Australia,"Jan 2-4, 1879",1879
3,England,"Sep 6-8, 1880",1880
4,drawn,"Dec 31, 1881 - Jan 4, 1882",1882


In [10]:
df['Year'] = pd.to_datetime(df['Year'],format='%Y')

In [11]:
df.drop(columns=['Date'], inplace=True)

In [12]:
#df.set_index('Year')

In [13]:
df.dropna(inplace=True)

In [14]:
df.shape

(2479, 2)

In [15]:
df.Team.value_counts()

drawn           786
Australia       409
England         384
West Indies     181
India           169
South Africa    169
Pakistan        145
New Zealand     107
Sri Lanka        95
Bangladesh       15
Zimbabwe         13
Afghanistan       3
tied              2
-                 1
Name: Team, dtype: int64

In [16]:
df = df[~df.Team.isin(['drawn', 'tied', '-'])]

In [17]:
df.Team.value_counts()

Australia       409
England         384
West Indies     181
India           169
South Africa    169
Pakistan        145
New Zealand     107
Sri Lanka        95
Bangladesh       15
Zimbabwe         13
Afghanistan       3
Name: Team, dtype: int64

In [18]:
df_test = df.groupby(['Year', 'Team']).size().groupby(level=1).cumsum().unstack().fillna(method='ffill')

In [19]:
df_test.fillna(0)

Team,Afghanistan,Australia,Bangladesh,England,India,New Zealand,Pakistan,South Africa,Sri Lanka,West Indies,Zimbabwe
Year,,,,,,,,,,,
1877-01-01,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1879-01-01,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1880-01-01,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1882-01-01,0.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1883-01-01,0.0,8.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2017-01-01,0.0,396.0,10.0,362.0,147.0,91.0,132.0,156.0,84.0,171.0,11.0
2018-01-01,0.0,399.0,13.0,370.0,154.0,95.0,136.0,162.0,88.0,174.0,12.0
2019-01-01,2.0,407.0,13.0,374.0,161.0,99.0,137.0,165.0,91.0,177.0,12.0


In [23]:
#bcr.bar_chart_race(df=df_test, title = 'Test Match Wins', period_fmt='%Y')

In [24]:
bcr.bar_chart_race(df=df_test, title = 'Test Match Wins', period_fmt='%Y', filename='Test_wins.mp4')